In [1]:
!nvidia-smi

Fri May 27 03:33:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. Install pre-requried library

In [2]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 29.9 MB/s 
     |████████████████████████████████| 86 kB 4.6 MB/s 
     |████████████████████████████████| 6.6 MB 40.7 MB/s 
     |████████████████████████████████| 596 kB 58.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!git clone https://github.com/huak95/JUTI.git

Cloning into 'JUTI'...
remote: Enumerating objects: 1636, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 1636 (delta 35), reused 141 (delta 24), pack-reused 1482
Receiving objects: 100% (1636/1636), 174.18 MiB | 16.60 MiB/s, done.
Resolving deltas: 100% (41/41), done.


# 2. Find background segmentation class 

In [4]:
import sys
sys.path.append('/content/JUTI')
%cd "/content/JUTI"

/content/JUTI


In [5]:
from utils.box_gen import *
from utils.path_help import *
from segment import FindLocation

import torch

### Start finding all class labels

In [6]:
fl = FindLocation()
all_labels = fl.get_sample_labels(0.2, "data/background/")

sample_size: 40
free mem: 15808069632
batch size: 50
Finished Pre-Segmentation


In [7]:
print("all_labels:", ", ".join(list(all_labels)))

all_labels: sky, ceiling, house, earth, signboard, fence, building, road, sidewalk, water, plant, door, palm, tree, wall


# 3. Select background class for each object

In [8]:
from segment import generate_data

gen = generate_data(all_labels, test=False)
sel_df = gen.get_selected_df()
sel_df

Selected bg label that 'cat' will fitted in [{'sky', 'ceiling', 'house', 'earth', 'signboard', 'fence', 'building', 'road', 'sidewalk', 'water', 'plant', 'door', 'palm', 'tree', 'wall'}]road
Selected bg label that 'dog' will fitted in [{'sky', 'ceiling', 'house', 'earth', 'signboard', 'fence', 'building', 'road', 'sidewalk', 'water', 'plant', 'door', 'palm', 'tree', 'wall'}]road
Selected bg label that 'raccoon' will fitted in [{'sky', 'ceiling', 'house', 'earth', 'signboard', 'fence', 'building', 'road', 'sidewalk', 'water', 'plant', 'door', 'palm', 'tree', 'wall'}]road


,fg,bg
0,cat,[road]
1,dog,[road]
2,raccoon,[road]


### Set backgound and object images size

In [9]:
gen.set_background_size(500)
gen.set_object_size(150)

Set Background Size to 500 px
Set Object Size to 150 px


# 4. Inference JUTI

In [10]:
from juti import generate_yolo_dataset_beta

In [12]:
# %%time
label_all = ['cat','dog','raccoon']
i = 2
print("i:", i)
label_select = label_all[i]

print("label_select:", label_select)
number_images = 10

generate_yolo_dataset_beta(gen,
                           f"data/background/",
                           f"data/object/{label_select}/",
                           n_images=number_images,
                           verbose=False,
                           )

i: 2
label_select: raccoon
free mem: 15780806656
batch size: 50


 20%|██        | 2/10 [00:00<00:00, 19.88it/s]

No Select Label -> Skip
Skipp


100%|██████████| 10/10 [00:00<00:00, 26.58it/s]

No Select Label -> Skip
Skipp
No Select Label -> Skip
Skipp
No Select Label -> Skip
Skipp
Finally Create raccoon
